In [ ]:
! pip install --upgrade codetiming numpy pandas census us mechanicalsoup geopandas pandas_bokeh torch
get_ipython().kernel.do_shutdown(True)

In [16]:
mount_path = '/content/drive'
repo_path = f'{mount_path}/MyDrive/gerrymandering/2022-10/voting_predictor'
from google.colab import drive
drive.mount(mount_path)
%cd {repo_path}
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload
from model import *
%cd {repo_path}
%load_ext google.colab.data_table

%load_ext autoreload
%autoreload
from model import *

class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size, activation):
        super(Feedforward, self).__init__()
        assert len(hidden_size) == len(activation), 'hidden_size and activation must have same length'
        L = []
        p = input_size
        for q, f in zip(hidden_size, activation):
            L.append(torch.nn.Linear(p, q))
            L.append(f())
            p = q
        self.nn = torch.nn.Sequential(*L)
        self.rmse_train = []
        self.rmse_test  = []

    def forward(self, X, W):
        self.prop = self.nn(X)
        # print(self.prop.shape, W.shape, B.shape)
        self.pred = ((self.prop * W) @ B).squeeze()
        # print(self.pred.shape)
        return self.pred

    def get_rmse(self):
        self.rmse_train.append(np.sqrt(loss_fcn(self(X_train, W_train), Y_train).item()))
        self.rmse_test .append(np.sqrt(loss_fcn(self(X_test , W_test ), Y_test ).item()))
        


feat = [
    # 'aland',
    # 'polsby_popper',
    'dist_to_border',
    # 'white_tot_pop'       , 'hisp_tot_pop'       , 'other_tot_pop',
    # 'white_vap_pop'       , 'hisp_vap_pop'       , 'other_vap_pop',
    'white_vap_density'   , 'hisp_vap_density'   , 'other_vap_density',
    'white_vap_poverty'   , 'hisp_vap_poverty'   , 'other_vap_poverty',
    'white_vap_elderly'   , 'hisp_vap_elderly'   , 'other_vap_elderly',
    'white_vap_highschool', 'hisp_vap_highschool', 'other_vap_highschool',
    'hisp_vap_spanish_at_home_english_well',
]
targ = ['d', 'r']
weig = ['white_vap_pop', 'hisp_vap_pop', 'other_vap_pop']

elections = ['2020_general_President', '2016_general_President', '2018_general_USSen', '2020_general_USSen']
df = pd.concat([features(*elec.split('_')) for elec in elections])[feat+targ+weig].sample(frac=1)
W = df[weig].astype(float)
W = W.join(W, lsuffix='_d', rsuffix='_r')
Y = df[targ].astype(float)
X = df[feat].astype(float)
X = (X - X.min()) / (X.max() - X.min())
B = torch.FloatTensor([[1,1,1,0,0,0],[0,0,0,1,1,1,]]).T.to(device)
for elec in elections:
    W_test  = torch.FloatTensor(W.query('election == @elec').values).to(device)
    X_test  = torch.FloatTensor(X.query('election == @elec').values).to(device)
    Y_test  = torch.FloatTensor(Y.query('election == @elec').values).to(device)
    W_train = torch.FloatTensor(W.query('election != @elec').values).to(device)
    X_train = torch.FloatTensor(X.query('election != @elec').values).to(device)
    Y_train = torch.FloatTensor(Y.query('election != @elec').values).to(device)

    model = Feedforward(
        input_size = X_train.shape[1],
        hidden_size = (50, 6),
        activation = (torch.nn.ReLU, torch.nn.Sigmoid)
    ).to(device)
    loss_fcn = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    steps = 1000
    for k in range(steps):
        optimizer.zero_grad()  # Clear gradient
        loss = loss_fcn(model(X_train, W_train), Y_train) # Compute train loss
        loss.backward()  # Backward propagation
        optimizer.step()  # Learn
        model.rmse_train.append(np.sqrt(loss.item()))
        # model.get_rmse()
    break
model.rmse_train[-10:]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/gerrymandering/2022-10/voting_predictor
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/content/drive/MyDrive/gerrymandering/2022-10/voting_predictor
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[223.69032025324654,
 223.64789919089336,
 223.60555742758274,
 223.5632862720308,
 223.5210333363507,
 223.47881611016288,
 223.4368531459168,
 223.39468995479726,
 223.35281612003462,
 223.31072452414818]

In [12]:
(model.pred-Y_train)/Y_train

RuntimeError: ignored